In [2]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.9 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 25.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 31.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.5/677.5 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [4]:
import psycopg2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
import os

In [5]:
import mlflow
import mlflow.sklearn

In [6]:
mlflow.set_tracking_uri("http://host.docker.internal:5000")  # URL donde corre MLflow

In [7]:
experiments = {exp.name: exp.experiment_id for exp in mlflow.search_experiments()}

if "random_forest_model" in experiments:
    mlflow.set_experiment(experiment_id=experiments["random_forest_model"])
else:
    mlflow.create_experiment("random_forest_model")
    mlflow.set_experiment("random_forest_model")

In [20]:
# conn = psycopg2.connect(
#     dbname='puj_mflow_models', user='postgres', password='postgres', host='db'
# )
# cursor = conn.cursor()

In [8]:
_data_root = './data/'

In [9]:
_data_filepath = os.path.join(_data_root, 'train_preprocessed.csv')

In [10]:
file_path = "./data/train_preprocessed.csv"
df = pd.read_csv(file_path)

In [11]:
# Separar características y etiquetas
X = df.drop(columns=["Cover_Type"])
y = df["Cover_Type"]

In [12]:
X = pd.get_dummies(X, drop_first=True)
X

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_C7755,Soil_Type_C7756,Soil_Type_C7757,Soil_Type_C7790,Soil_Type_C8703,Soil_Type_C8707,Soil_Type_C8708,Soil_Type_C8771,Soil_Type_C8772,Soil_Type_C8776
0,3305,35,13,134,25,5811,218,211,127,659,...,False,False,False,False,False,False,False,True,False,False
1,3295,332,10,934,160,924,196,227,170,2519,...,False,True,False,False,False,False,False,False,False,False
2,3320,339,12,711,106,979,195,223,167,2531,...,False,False,False,False,False,False,False,False,False,False
3,3321,16,14,785,193,3221,207,209,137,787,...,False,False,False,False,False,False,False,False,False,False
4,2959,233,17,190,85,1519,189,253,198,999,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92957,2324,26,33,124,67,1473,185,150,81,1034,...,False,False,False,False,False,False,False,False,False,False
92958,2384,144,6,95,10,604,230,239,143,934,...,False,False,False,False,False,False,False,False,False,False
92959,3287,96,6,67,4,4496,231,231,133,3684,...,False,False,False,False,False,False,False,False,False,False
92960,2960,344,8,30,3,3193,205,228,162,1176,...,False,False,False,False,False,False,False,False,False,False


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Definir hiperparámetros a probar
n_estimators_list = [10, 50, 100, 200]
max_depth_list = [None, 5, 10, 20]

In [18]:
for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        with mlflow.start_run():  # Iniciar un experimento en MLflow
            # Crear y entrenar modelo
            model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)

            # # Guardar resultados en la base de datos
            # cursor.execute(
            #     "INSERT INTO experiment_results (n_estimators, max_depth, accuracy) VALUES (%s, %s, %s)",
            #     (n_estimators, max_depth, acc)
            # )
            # conn.commit()

            # Registrar parámetros y métricas en MLflow
            mlflow.log_param("n_estimators", n_estimators)
            mlflow.log_param("max_depth", max_depth)
            mlflow.log_metric("accuracy", acc)
            mlflow.sklearn.log_model(model, "random_forest_model")

            print(f'n_estimators={n_estimators}, max_depth={max_depth}, Accuracy={acc:.5f}')


2025/03/18 06:58:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run sassy-shad-446 at: http://host.docker.internal:5000/#/experiments/1/runs/beb84b3cf57e46f9a07ea82c0b42db3d
🧪 View experiment at: http://host.docker.internal:5000/#/experiments/1


PermissionError: [Errno 13] Permission denied: '/home/d4n1el'

In [32]:
# Entrenamiento del modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [33]:
# Predicción y evaluación
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Precisión del modelo: {accuracy:.4f}")
print(f"Precisión: {precision:.4f}")
print(f"Sensibilidad (Recall): {recall:.4f}")
print(f"Puntaje F1: {f1:.4f}")

Precisión del modelo: 0.9024
Precisión: 0.9028
Sensibilidad (Recall): 0.9024
Puntaje F1: 0.9012
